### 1 - Import test data

In [21]:
import yfinance as yf
import pandas as pd

dataF = yf.download("EURUSD=X", start="2023-9-15", end="2023-11-1", interval='15m')
dataF.iloc[:,:]
#dataF.Open.iloc

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2023-09-15 00:00:00,1.064509,1.064509,1.064169,1.064283,1.064283,0
2023-09-15 00:15:00,1.064283,1.064283,1.064169,1.064283,1.064283,0
2023-09-15 00:30:00,1.064283,1.064283,1.064169,1.064169,1.064169,0
2023-09-15 00:45:00,1.064056,1.064056,1.063717,1.063830,1.063830,0
2023-09-15 01:00:00,1.063830,1.064623,1.063830,1.064509,1.064509,0
...,...,...,...,...,...,...
2023-10-31 22:45:00,1.058089,1.058313,1.058089,1.058201,1.058201,0
2023-10-31 23:00:00,1.058313,1.058313,1.058089,1.058089,1.058089,0
2023-10-31 23:15:00,1.058089,1.058201,1.058089,1.058089,1.058089,0


### 2 - Define your signal function

In [22]:
def signal_generator(df):
    open = df.Open.iloc[-1]
    close = df.Close.iloc[-1]
    previous_open = df.Open.iloc[-2]
    previous_close = df.Close.iloc[-2]

    # Bearish Pattern
    if (open>close and
    previous_open<previous_close and
    close<previous_open and
    open>=previous_close):
        return 1

    # Bullish Pattern
    elif (open<close and
        previous_open>previous_close and
        close>previous_open and
        open<=previous_close):
        return 2

    # No clear pattern
    else:
        return 0

signal = []
signal.append(0)
for i in range(1,len(dataF)):
    df = dataF[i-1:i+1]
    signal.append(signal_generator(df))
#signal_generator(data)
dataF["signal"] = signal

In [23]:
dataF.signal.value_counts()
#dataF.iloc[:, :]

signal
0    2864
1     135
2     127
Name: count, dtype: int64

### 3 - Connect to the market and execute trades

In [24]:
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

In [25]:
access_token= "cd7e21b015cb734770f5fcfea9da9c9f-49a95de76ed5cab29ee0d33cee5f004d"
accountID = "101-001-27332384-001" #your account ID here
list_of_currency = [Pair.EUR_USD, Pair.EUR_GBP, Pair.USD_JPY, Pair.AUD_USD, Pair.USD_CAD, Pair.CAD_JPY, Pair.NZD_USD]
def get_candles(n, currency):
    client = CandleClient(access_token, real=False)
    collector = client.get_collector(currency, Gran.M15)
    candles = collector.grab(n)
    return candles

candles = get_candles(3, Pair.EUR_USD)
for candle in candles:
    print(float(str(candle.bid.o))>1)


True
True
True


In [26]:
def trading_job(currency, string_currency):
    candles = get_candles(3, currency)
    dfstream = pd.DataFrame(columns=['Open','Close','High','Low'])
    i=0
    for candle in candles:
        dfstream.loc[i, ['Open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['Close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['High']] = float(str(candle.bid.h))
        dfstream.loc[i, ['Low']] = float(str(candle.bid.l))
        i=i+1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    signal = signal_generator(dfstream.iloc[:-1,:])#
    # EXECUTING ORDERS
    print("signal: " + str(signal))
    client = API(access_token)

    SLTPRatio = 2.
    previous_candleR = abs(dfstream['High'].iloc[-2]-dfstream['Low'].iloc[-2])

    SLBuy = float(str(candle.bid.o))-previous_candleR
    SLSell = float(str(candle.bid.o))+previous_candleR

    TPBuy = float(str(candle.bid.o))+previous_candleR*SLTPRatio
    TPSell = float(str(candle.bid.o))-previous_candleR*SLTPRatio

    #print(dfstream.iloc[:-1,:])
    #print(TPBuy, "  ", SLBuy, "  ", TPSell, "  ", SLSell)
    #signal = 2
    #Sell
    if signal == 1:
        mo = MarketOrderRequest(instrument=string_currency, units=-25000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)
    #Buy
    elif signal == 2:
        mo = MarketOrderRequest(instrument=string_currency, units=25000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

In [28]:
list_of_currency = [Pair.EUR_USD, Pair.EUR_GBP, Pair.USD_JPY, Pair.AUD_USD, Pair.USD_CAD, Pair.CAD_JPY, Pair.NZD_USD]
list_of_str_currency = ["EUR_USD", "EUR_GBP", "USD_JPY", "AUD_USD","USD_CAD","CAD_JPY","NZD_USD"]

# for i in range (len(list_of_str_currency)):
#     trading_job(list_of_currency[i],list_of_str_currency[i])


scheduler = BlockingScheduler()
for i in range (len(list_of_str_currency)):
    scheduler.add_job(trading_job, 'cron', args = [list_of_currency[i],list_of_str_currency[i]], day_of_week='mon-fri', hour='00-23', minute='1,11,21,31,41,51', start_date='2022-01-12 12:00:00')
scheduler.start()

signal: 0signal: 0

signal: 0
signal: 0
signal: 0
signal: 0
signal: 0
signal: 0signal: 0

signal: 0
signal: 0
signal: 0
signal: 2
signal: 0
{'orderCreateTransaction': {'id': '316', 'accountID': '101-001-27332384-001', 'userID': 27332384, 'batchID': '316', 'requestID': '61170934764682523', 'time': '2023-11-07T19:31:00.407706028Z', 'type': 'MARKET_ORDER', 'instrument': 'CAD_JPY', 'units': '25000', 'timeInForce': 'FOK', 'positionFill': 'DEFAULT', 'takeProfitOnFill': {'price': '109.462', 'timeInForce': 'GTC'}, 'stopLossOnFill': {'price': '109.288', 'timeInForce': 'GTC', 'triggerMode': 'TOP_OF_BOOK'}, 'reason': 'CLIENT_ORDER'}, 'orderFillTransaction': {'id': '317', 'accountID': '101-001-27332384-001', 'userID': 27332384, 'batchID': '316', 'requestID': '61170934764682523', 'time': '2023-11-07T19:31:00.407706028Z', 'type': 'ORDER_FILL', 'orderID': '316', 'instrument': 'CAD_JPY', 'units': '25000', 'requestedUnits': '25000', 'price': '109.371', 'pl': '0.0000', 'quotePL': '0', 'financing': '0.00

Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,11,21,31,41,51'], next run at: 2023-11-07 13:21:00 PST)" was missed by 0:06:34.217327
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,11,21,31,41,51'], next run at: 2023-11-07 13:21:00 PST)" was missed by 0:06:34.218906
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,11,21,31,41,51'], next run at: 2023-11-07 13:21:00 PST)" was missed by 0:06:34.219094
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,11,21,31,41,51'], next run at: 2023-11-07 13:21:00 PST)" was missed by 0:06:34.221615
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,11,21,31,41,51'], next run at: 2023-11-07 13:21:00 PST)" was missed by 0:06:34.221769
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,11,21,31,41,51'], next run at:

signal: 0signal: 0
signal: 0

signal: 0
signal: 0
signal: 0
signal: 0
signal: 0
signal: 0
signal: 0
signal: 0
signal: 0
signal: 0
signal: 0
